In [ ]:
!pip install -qU langchain langchain-community langchain-google-genai chromadb pypdf sentence-transformers opentelemetry-sdk opentelemetry-api

In [ ]:
import sys
import chromadb
import langchain
from langchain_google_genai import GoogleGenerativeAIEmbeddings

try:
    from opentelemetry.sdk._logs import ReadableLogRecord
    print("Environment check passed: ChromaDB and OpenTelemetry are synced.")
except ImportError:
    print("Environment check failed: Please restart the session and try again.")

print(f"Python version: {sys.version}")

In [ ]:
!pip install -qU langchain-text-splitters

In [ ]:
!pip install -qU langchain-classic

In [ ]:
import os
from google.colab import userdata

# Standard Modular Imports
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI

# Legacy Imports (Now in langchain-classic)
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

print("✅ All modules imported successfully!")

In [ ]:
os.environ["GOOGLE_API_KEY"] = ""

In [ ]:
!mkdir sacco_docs


In [ ]:
from langchain_community.document_loaders import DirectoryLoader

In [ ]:
loader = DirectoryLoader('./sacco_docs/', glob="./*.pdf", loader_cls=PyPDFLoader)
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)
print(f"Successfully split policy into {len(chunks)} searchable chunks.")



In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./sacco_db"
)


In [ ]:
!pip install  langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=0)

In [ ]:
from langchain_classic.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_db.as_retriever(search_kwargs={"k": 5})
)

print("Bot initialized successfully with Gemini 3 Flash Preview!")

In [ ]:
def ask_sacco_bot(question):
    print(f"\nQuestion: {question}")
    response = qa_chain.invoke(question)
    return response["result"]


In [ ]:
# EXAMPLE USAGE:
question = input("What question do you have about the SACCO? ")
response = ask_sacco_bot(question)
print(f"Bot Answer: {response}")


In [ ]:
import json

# 1. Path to your current notebook file
# If you just uploaded it to the sidebar, it's likely '/content/FAQChatbot.ipynb'
input_file = 'FAQChatbot.ipynb'
output_file = 'FAQChatbot_Fixed.ipynb'

# 2. Load the notebook data
with open(input_file, 'r', encoding='utf-8') as f:
    nb_data = json.load(f)

# 3. Remove the problematic widgets metadata that GitHub hates
if 'metadata' in nb_data and 'widgets' in nb_data['metadata']:
    del nb_data['metadata']['widgets']
    print("Found and removed 'widgets' metadata.")
else:
    print("No 'widgets' metadata found, or it was already removed.")

# 4. Save the fixed notebook
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(nb_data, f, indent=2)

print(f"Success! Download '{output_file}' from the file sidebar and upload it to GitHub.")